# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos


def f(a, b, c):
    return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5


print(f(2, 3, 4))

6.336362190988558


$$f(a,b,c) = -a^3 + \sin(3b) - \frac{1}{c} + b^{2.5} - a^{0.5}$$
$$\frac{\partial f}{\partial a} = -3a^2 - 0.5 a^{-0.5} $$
$$\frac{\partial f}{\partial b} = 3\cos(3b) + 2.5 b^{1.5}$$
$$\frac{\partial f}{\partial c} =  +\frac{1}{c^2}$$

In [2]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
    # todo, return [df/da, df/db, df/dc]
    return [-3*a**2 - 0.5*a**(-0.5), 3*cos(3*b) + 2.5*b**(1.5), 1/c**2]


# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
    ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(
        f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


Using the defnition of the derivative:
$$
\frac{\partial f}{\partial a} = \lim_{h\rightarrow0} \frac{f(a+h,b,c) - f(a,b,c)}{h}
$$

In [3]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
h = 1e-6
a, b, c = 2, 3, 4
f1 = f(a, b, c)
numerical_grad = [(f(a+h, b, c)-f1)/h, (f(a, b+h, c)-f1) /
                  h, (f(a, b, c+h)-f1)/h]  # TODO
# -----------

for dim in range(3):
    ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(
        f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [4]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
h = 1e-6
a, b, c = 2, 3, 4
numerical_grad2 = [(f(a+h, b, c)-f(a-h, b, c))/(2*h), (f(a, b+h, c) -
                                                       f(a, b-h, c))/(2*h), (f(a, b, c+h)-f(a, b, c-h))/(2*h)]  # TODO
# -----------

for dim in range(3):
    ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(
        f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


## section 2: support for softmax

In [5]:
# Value class starter code, with many functions taken out
from math import exp, log


class Value:

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):  # exactly as in the video
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    # ------
    # re-implement all the other functions needed for the exercises below
    # your code here
    # TODO

    def exp(self):
        out = Value(exp(self.data), (self,), 'e^')

        def _backward():
            self.grad += out * out.grad

        out._backward = _backward
        return out

    # def __pow__(self, other):
    #     # only int or float powers for now for some reason not value
    #     assert isinstance(other, (int, float))
    #     out = Value(self.data**other, (self,), f'^{other}')

    #     def _backward():
    #         self.grad += other * self.data ** (other - 1) * out.grad
    #     out._backward = _backward
    #     return out

    # def __truediv__(self, other):
    #     return self * other**-1

    def __truediv__(self, other):
        out = Value(self.data / other.data, (self, other), '/')

        def _backward():
            self.grad += (1/other.data) * out.grad
            other.grad += -(self.data/other.data**2) * out.grad

        out._backward = _backward
        return out

    def __neg__(self):
        return -1 * self

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data*other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __rmul__(self, other):
        return self * other

    def __sub__(self, other):
        return self + (-other)

    def __abs__(self):
        out = Value(abs(self.data), (self,), '||')

        def _backward():
            if self.data >= 0:
                self.grad += out.grad
            self.grad += -1 * out.grad

        out._backward = _backward
        return out

    def __lt__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return self.data < other.data

    def log(self):
        out = Value(log(self.data), (self,), 'log')

        def _backward():
            self.grad += (1/self.data) * out.grad

        out._backward = _backward
        return out

    def __radd__(self, other):
        return self + other

    # ------

    def backward(self):  # exactly as in video
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

Softmax:
$$
\sigma({\bf a})_i = \frac{e^{a_i}}{\sum e^{a_j}}
$$
Derivative of softmax:
$$
\frac{\partial}{\partial a_k} \sigma({\bf a})_i = \frac{\sum e^{a_j} e^{a_i}\delta_{ik} - e^{a_i} e^{a_k}}{\left(\sum e^{a_j}\right)^2} = \frac{e^{a_i}}{\sum e^{a_j}} \frac{\sum e^{a_j} \delta_{ik} - e^{a_k}}{\sum e^{a_j}} = \sigma({\bf a})_i \left(\delta_{ik} - \sigma({\bf a})_k\right)
$$

In [6]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
    counts = [logit.exp() for logit in logits]
    _test_counts = [exp(logit.data) for logit in logits]
    print('counts', counts, _test_counts)
    denominator = sum(counts)
    _test_denominator = sum(_test_counts)
    print('denom', denominator, _test_denominator)
    out = [c / denominator for c in counts]
    _test_out = [c / _test_denominator for c in _test_counts]
    print('softmax', out, _test_out)
    return out


# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log()  # dim 3 acts as the label for this input example
_test_loss = -log(probs[3].data)
print('loss', loss, _test_loss)
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319,
       0.005653302662216329, -0.8864503806400986]
for dim in range(4):
    ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
    print(
        f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

counts [Value(data=1.0), Value(data=20.085536923187668), Value(data=0.1353352832366127), Value(data=2.718281828459045)] [1.0, 20.085536923187668, 0.1353352832366127, 2.718281828459045]
denom Value(data=23.939154034883327) 23.939154034883327
softmax [Value(data=0.04177257051535045), Value(data=0.839024507462532), Value(data=0.005653302662216329), Value(data=0.11354961935990121)] [0.04177257051535045, 0.839024507462532, 0.005653302662216329, 0.11354961935990121]
loss Value(data=2.1755153626167147) 2.1755153626167147
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns Value(data=0.041772570515350445)
OK for dim 1: expected 0.8390245074625319, yours returns Value(data=0.8390245074625319)
OK for dim 2: expected 0.005653302662216329, yours returns Value(data=0.005653302662216329)
OK for dim 3: expected -0.8864503806400986, yours returns Value(data=-0.886450380640099)


In [45]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
logits = torch.tensor([0, 3, -2, 1], requires_grad=True, dtype=torch.float64)
print(logits)
probs = torch.nn.functional.softmax(logits, dim=0)
loss = -probs[3].log()
loss.backward()
print(loss.data)
print(logits.grad)

ans = [0.041772570515350445, 0.8390245074625319,
       0.005653302662216329, -0.8864503806400986]
my_ans = logits.grad.tolist()
print('my_ans', my_ans)
for dim in range(4):
    ok = 'OK' if abs(my_ans[dim] - ans[dim]) < 1e-5 else 'WRONG!'
    print(
        f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits.grad[dim].item()}")

tensor([ 0.,  3., -2.,  1.], dtype=torch.float64, requires_grad=True)
tensor(2.1755, dtype=torch.float64)
tensor([ 0.0418,  0.8390,  0.0057, -0.8865], dtype=torch.float64)
my_ans [0.04177257051535046, 0.8390245074625321, 0.00565330266221633, -0.8864503806400987]
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535046
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625321
OK for dim 2: expected 0.005653302662216329, yours returns 0.00565330266221633
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400987
